In [119]:
import pandas as pd

# RAIS

## Estabelecimentos


TODO

- Pipeline

    - Leitura
    - Tratamento
    - Merging
    - Load

ano, uf, meso, micro, muni

In [120]:
def read_csv_file(caminho, ano):
    try: # txt file
        colunas = ['CNAE 2.0 Classe', 'Município']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.txt', sep=';', encoding='latin1', usecols=colunas)
        df_estb['ano'] = ano
        df_estb.rename(columns={'Município': 'id_municipio', 'CNAE 2.0 Classe': 'cnae_2'}, inplace=True)	
        df_estb = df_estb[['ano', 'id_municipio', 'cnae_2']]
    except: # csv file
        colunas = ['ano', 'id_municipio', 'cnae_2']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.csv', usecols=colunas)
        df_estb['id_municipio'] = df_estb['id_municipio'].apply(lambda x: int(x/10)) # remove 7'th dígito

    return df_estb

In [121]:
def transform_data(df):
    '''Normaliza o tipo de cada coluna e obtem a cnae_secao de cnae_2'''
    
    df['ano'] = df['ano'].astype('int16')
    
    df['id_municipio'] = df['id_municipio'].astype('int')
        
    df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))
    df['cnae_secao'] = df['cnae_2'].apply(lambda x: x[:2])
    df['cnae_secao'] = df['cnae_secao'].astype('int')
    df.drop(columns=['cnae_2'], inplace=True)
    return df

In [122]:
def merge_municipios(df, caminho):
    '''Faz o merge do id_municipio com os dados do dicionário'''
    
    colunas_muni = ['id_municipio_6', 'nome', 'nome_microrregiao', 'nome_mesorregiao', 'sigla_uf', 'nome_uf', 'nome_regiao']
    dicionario_muni = pd.read_csv(caminho, sep=',', usecols=colunas_muni)
    dicionario_muni.rename(columns={'id_municipio_6': 'id_municipio', 'nome': 'municipio'}, inplace=True)
    
    df = pd.merge(df, dicionario_muni, on='id_municipio', how='left')
    df.drop(columns=['id_municipio'], inplace=True)
    return df

In [123]:
def merge_cnae(df, caminho):
    
    colunas_tmp = ['divisao','descricao_secao']
    df_cnae = pd.read_csv(caminho, usecols=colunas_tmp)
    df_cnae.rename(columns={'divisao': 'cnae_secao'}, inplace=True)
    df_cnae.drop_duplicates(inplace=True)
    
    df = pd.merge(df, df_cnae, on='cnae_secao', how='left')
    df.drop(columns=['cnae_secao'], inplace=True)
    return df

In [ ]:
def etl_pipeline(caminhos, ano_ini, ano_fim):
    for ano in range(ano_ini, ano_fim+1):
        df = read_csv_file(caminhos['rais'], str(ano))
        df = transform_data(df)
        df = merge_municipios(df, caminhos['municipios'])
        df = merge_cnae(df, caminhos['cnae'])
        
    return df

In [125]:
caminho = "D:/dados/rais/estabelecimentos/"
caminho_dict_muni = "D:/dados/rais/dicionarios/dicionario_municipios.csv"
caminho_dict_cnae = "D:/dados/rais/dicionarios/dicionario_cnae_2.csv"
ano_ini = 2010
ano_fim = 2015
caminhos = {'rais': caminho, 
            'municipios': caminho_dict_muni, 
            'cnae': caminho_dict_cnae}

df = etl_pipeline(caminhos, ano_ini, ano_fim)
df

C:\Users\ramos\AppData\Local\Temp\ipykernel_4200\2426591515.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['94995' '47890' '65201' ... '01512' '69117' '01113']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))
C:\Users\ramos\AppData\Local\Temp\ipykernel_4200\2426591515.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['01113' '01113' '01113' ... '85996' '78205' '81214']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))
C:\Users\ramos\AppData\Local\Temp\ipykernel_4200\2426591515.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['23303' '47547' '86305'

,ano,municipio,nome_microrregiao,nome_mesorregiao,sigla_uf,nome_uf,nome_regiao,descricao_secao
0,2010,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste,Outras Atividades De Serviços
1,2010,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste,Comércio; Reparação De Veículos Automotores E ...
2,2010,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste,"Atividades Financeiras, De Seguros E Serviços ..."
3,2010,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste,Outras Atividades De Serviços
4,2010,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste,Atividades Administrativas E Serviços Compleme...
...,...,...,...,...,...,...,...,...
48124343,2015,Porto Alegre,Porto Alegre,Metropolitana de Porto Alegre,RS,Rio Grande do Sul,Sul,Outras Atividades De Serviços
48124344,2015,Porto Alegre,Porto Alegre,Metropolitana de Porto Alegre,RS,Rio Grande do Sul,Sul,Outras Atividades De Serviços
48124345,2015,Garibaldi,Caxias do Sul,Nordeste Rio-grandense,RS,Rio Grande do Sul,Sul,Outras Atividades De Serviços
48124346,2015,Santa Maria,Santa Maria,Centro Ocidental Rio-grandense,RS,Rio Grande do Sul,Sul,Outras Atividades De Serviços


---

In [7]:
df_estb.loc[:, 'CNAE 2.0 Classe'] = df_estb['CNAE 2.0 Classe'].apply(lambda x: str(x).zfill(5))
df_estb['cnae_secao'] = df_estb['CNAE 2.0 Classe'].apply(lambda x: x[:2])
df_estb['cnae_secao'] = df_estb['cnae_secao'].astype('category')

df_estb.info()

C:\Users\ramos\AppData\Local\Temp\ipykernel_1848\1264700111.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['47741' '25128' '47318' ... '47512' '94308' '68102']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_estb.loc[:, 'CNAE 2.0 Classe'] = df_estb['CNAE 2.0 Classe'].apply(lambda x: str(x).zfill(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8240846 entries, 0 to 8240845
Data columns (total 6 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   CNAE 2.0 Classe          object  
 1   Ind Atividade Ano        int64   
 2   id_municipio_6           int64   
 3   Tamanho Estabelecimento  int64   
 4   UF                       int64   
 5   cnae_secao               category
dtypes: category(1), int64(4), object(1)
memory usage: 322.2+ MB


In [ ]:
def clean_data(df):
    '''Change type, split and rename columns'''
    
    df = df.astype({'Unnamed: 2': 'string'})
    loc_0 = df.columns.get_loc('Unnamed: 2')
    df_split = df['Unnamed: 2'].str.split(pat=' \\.\\. ', expand=True).add_prefix('Unnamed: 2_')
    df = pd.concat([df.iloc[:, :loc_0], df_split, df.iloc[:, loc_0:]], axis=1)
    df = df.drop(columns=['Unnamed: 2'])
    
    df.rename(columns={'Unnamed: 2_0': 'cat_min',
                       'Unnamed: 2_1': 'cat_max',
                       'Unnamed: 4': 'denominacao'},
              inplace=True)
    return df

def map_cnae_cat(df_estb, cat_dict):
    '''mapeia a seção CNAE 2.0 para a denominação da categoria'''
    
    df_estb['cnae_secao'] = df_estb['cnae_secao'].astype(int).map(cat_dict)
    df_estb.drop(columns=['CNAE 2.0 Classe'], inplace=True)
    return df_estb

,2.1 - Grandes categorias,cat_min,cat_max,denominacao
4,A,01,03,"Agricultura, pecuária, produção florestal, pes..."
5,B,05,09,Indústrias extrativas
6,C,10,33,Indústrias de transformação
7,D,35,<NA>,Eletricidade e gás
8,E,36,39,"Água, esgoto, atividades de gestão de resíduos..."
9,F,41,43,Construção
10,G,45,47,Comércio; reparação de veículos automotores e ...
11,H,49,53,"Transporte, armazenagem e correio"
12,I,55,56,Alojamento e alimentação
13,J,58,63,Informação e comunicação


Mapeia CNAE para dua denominação

In [ ]:
categorias = pd.read_excel('dados_extraidos/cnae2_0/categorias.xls',
                           usecols=['2.1 - Grandes categorias', 'Unnamed: 2', 'Unnamed: 4'],)

df_cnae = clean_data(categorias.copy().loc[4:,:])

minimo = df_cnae['cat_min'].to_list()
maximo = df_cnae['cat_max'].to_list()
cat = df_cnae['denominacao'].to_list()
cat_dict = {}

for min, max, cat in zip(minimo, maximo, cat):  # hash de categorias cnae
    if type(max) == type(pd.NA):
        max = min
    for i in range(int(min), int(max)+1):
        cat_dict[i] = cat
        
map_cnae_cat(df_estb, cat_dict)
df_estb

,Ind Atividade Ano,id_municipio_6,Tamanho Estabelecimento,UF,cnae_secao
0,1,110030,3,11,Comércio; reparação de veículos automotores e ...
1,1,110020,3,11,Indústrias de transformação
2,1,110037,2,11,Comércio; reparação de veículos automotores e ...
3,1,110012,2,11,Comércio; reparação de veículos automotores e ...
4,1,110012,2,11,Alojamento e alimentação
...,...,...,...,...,...
8240841,0,530010,1,53,Informação e comunicação
8240842,1,530010,1,53,Construção
8240843,1,530010,1,53,Comércio; reparação de veículos automotores e ...
8240844,1,530010,1,53,Outras atividades de serviços


Mapeia id_municipio para suas respectivas regiões

In [14]:
colunas_muni = ['id_municipio_6', 'nome', 'nome_microrregiao', 'nome_mesorregiao', 'sigla_uf', 'nome_uf', 'nome_regiao']
dicionario_muni = pd.read_csv('dados_extraidos/rais/dicionarios/dicionario_municipios.csv', sep=',', usecols=colunas_muni)
df = pd.merge(df_estb, dicionario_muni, on='id_municipio_6', how='left')
df

,Ind Atividade Ano,id_municipio_6,Tamanho Estabelecimento,UF,cnae_secao,nome,nome_microrregiao,nome_mesorregiao,sigla_uf,nome_uf,nome_regiao
0,1,110030,3,11,Comércio; reparação de veículos automotores e ...,Vilhena,Vilhena,Leste Rondoniense,RO,Rondônia,Norte
1,1,110020,3,11,Indústrias de transformação,Porto Velho,Porto Velho,Madeira-Guaporé,RO,Rondônia,Norte
2,1,110037,2,11,Comércio; reparação de veículos automotores e ...,Alto Alegre dos Parecis,Cacoal,Leste Rondoniense,RO,Rondônia,Norte
3,1,110012,2,11,Comércio; reparação de veículos automotores e ...,Ji-Paraná,Ji-Paraná,Leste Rondoniense,RO,Rondônia,Norte
4,1,110012,2,11,Alojamento e alimentação,Ji-Paraná,Ji-Paraná,Leste Rondoniense,RO,Rondônia,Norte
...,...,...,...,...,...,...,...,...,...,...,...
8240841,0,530010,1,53,Informação e comunicação,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste
8240842,1,530010,1,53,Construção,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste
8240843,1,530010,1,53,Comércio; reparação de veículos automotores e ...,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste
8240844,1,530010,1,53,Outras atividades de serviços,Brasília,Brasília,Distrito Federal,DF,Distrito Federal,Centro-Oeste


Cálculos QL

In [27]:
df_grouped = df.groupby(['nome_mesorregiao', 'cnae_secao']).size().reset_index(name='count')
df_grouped.query('nome_mesorregiao == "Norte de Minas"')

,nome_mesorregiao,cnae_secao,count
1701,Norte de Minas,"Administração pública, defesa e seguridade social",283
1702,Norte de Minas,"Agricultura, pecuária, produção florestal, pes...",4992
1703,Norte de Minas,Alojamento e alimentação,2368
1704,Norte de Minas,"Artes, cultura, esporte e recreação",501
1705,Norte de Minas,Atividades administrativas e serviços compleme...,1995
1706,Norte de Minas,"Atividades financeiras, de seguros e serviços ...",416
1707,Norte de Minas,Atividades imobiliárias,251
1708,Norte de Minas,"Atividades profissionais, científicas e técnicas",1534
1709,Norte de Minas,Comércio; reparação de veículos automotores e ...,23843
1710,Norte de Minas,Construção,2210


---

Teste

In [15]:
colunas = ['CNAE 2.0 Classe', 'Município']
tmp = pd.read_csv("D:/dados/rais/estabelecimentos/ESTB2007.txt", encoding='latin1', sep=';', usecols=colunas)
tmp

,CNAE 2.0 Classe,Município
0,64221,110020
1,64221,110010
2,64221,110012
3,64221,110002
4,64221,110004
...,...,...
6887953,81125,530010
6887954,73114,530010
6887955,71111,530010
6887956,81125,530010


In [16]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6887958 entries, 0 to 6887957
Data columns (total 2 columns):
 #   Column           Dtype
---  ------           -----
 0   CNAE 2.0 Classe  int64
 1   Município        int64
dtypes: int64(2)
memory usage: 105.1 MB
